In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [4]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [5]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns
            ),
            ignore_index = True
        )
        

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.79,N/A,-0.091793,N/A,-0.217167,N/A,-0.100269,N/A,-0.008923,N/A,N/A
1,AAL,16.68,N/A,-0.311946,N/A,-0.173035,N/A,-0.076653,N/A,-0.198014,N/A,N/A
2,AAP,190.41,N/A,0.007119,N/A,-0.204364,N/A,-0.094145,N/A,-0.174214,N/A,N/A
3,AAPL,146.86,N/A,0.117349,N/A,-0.133375,N/A,-0.149284,N/A,-0.13618,N/A,N/A
4,ABBV,156.70,N/A,0.391528,N/A,0.309249,N/A,0.02501,N/A,-0.028564,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,121.52,N/A,-0.026677,N/A,-0.086797,N/A,-0.068775,N/A,-0.048355,N/A,N/A
497,ZBH,123.08,N/A,-0.290066,N/A,-0.083847,N/A,-0.05779,N/A,-0.069814,N/A,N/A
498,ZBRA,336.68,N/A,-0.373253,N/A,-0.462302,N/A,-0.236606,N/A,-0.177044,N/A,N/A
499,ZION,56.00,N/A,0.005937,N/A,-0.175768,N/A,-0.229763,N/A,-0.096576,N/A,N/A


In [6]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        if hqm_dataframe.loc[row,change_col] == None:
            hqm_dataframe = hqm_dataframe.drop(row)
            break;
hqm_dataframe.reset_index(drop = True, inplace = True)

            
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

            

hqm_dataframe



,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.79,N/A,-0.091793,0.394366,-0.217167,0.283702,-0.100269,0.432596,-0.008923,0.716298,N/A
1,AAL,16.68,N/A,-0.311946,0.092555,-0.173035,0.368209,-0.076653,0.49497,-0.198014,0.066398,N/A
2,AAP,190.41,N/A,0.007119,0.60161,-0.204364,0.307847,-0.094145,0.444668,-0.174214,0.094567,N/A
3,AAPL,146.86,N/A,0.117349,0.750503,-0.133375,0.458753,-0.149284,0.289738,-0.13618,0.183099,N/A
4,ABBV,156.70,N/A,0.391528,0.93159,0.309249,0.935614,0.02501,0.72837,-0.028564,0.62173,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,121.52,N/A,-0.026677,0.529175,-0.086797,0.54326,-0.068775,0.513078,-0.048355,0.517103,N/A
493,ZBH,123.08,N/A,-0.290066,0.112676,-0.083847,0.549296,-0.05779,0.559356,-0.069814,0.412475,N/A
494,ZBRA,336.68,N/A,-0.373253,0.054326,-0.462302,0.018109,-0.236606,0.108652,-0.177044,0.086519,N/A
495,ZION,56.00,N/A,0.005937,0.597586,-0.175768,0.362173,-0.229763,0.122736,-0.096576,0.301811,N/A


In [7]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.79,N/A,-0.091793,0.394366,-0.217167,0.283702,-0.100269,0.432596,-0.008923,0.716298,0.45674
1,AAL,16.68,N/A,-0.311946,0.092555,-0.173035,0.368209,-0.076653,0.49497,-0.198014,0.066398,0.255533
2,AAP,190.41,N/A,0.007119,0.60161,-0.204364,0.307847,-0.094145,0.444668,-0.174214,0.094567,0.362173
3,AAPL,146.86,N/A,0.117349,0.750503,-0.133375,0.458753,-0.149284,0.289738,-0.13618,0.183099,0.420523
4,ABBV,156.70,N/A,0.391528,0.93159,0.309249,0.935614,0.02501,0.72837,-0.028564,0.62173,0.804326
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,121.52,N/A,-0.026677,0.529175,-0.086797,0.54326,-0.068775,0.513078,-0.048355,0.517103,0.525654
493,ZBH,123.08,N/A,-0.290066,0.112676,-0.083847,0.549296,-0.05779,0.559356,-0.069814,0.412475,0.408451
494,ZBRA,336.68,N/A,-0.373253,0.054326,-0.462302,0.018109,-0.236606,0.108652,-0.177044,0.086519,0.066901
495,ZION,56.00,N/A,0.005937,0.597586,-0.175768,0.362173,-0.229763,0.122736,-0.096576,0.301811,0.346076


In [8]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,OXY,67.900,N/A,1.766426,0.995976,1.093732,1.0,0.723037,1.0,0.215802,0.987928,0.995976
1,DVN,73.831,N/A,2.019952,0.997988,0.658945,0.987928,0.331095,0.985915,0.281611,1.0,0.992958
2,MRO,31.379,N/A,1.570223,0.993964,0.762104,0.993964,0.345425,0.987928,0.209224,0.983903,0.98994
3,VLO,128.950,N/A,0.729904,0.967807,0.786144,0.995976,0.535465,0.997988,0.279869,0.997988,0.98994
4,APA,48.346,N/A,1.353147,0.991952,0.627307,0.985915,0.361613,0.991952,0.172656,0.971831,0.985412
5,COP,115.000,N/A,1.115915,0.98994,0.520268,0.973843,0.234812,0.961771,0.221167,0.98994,0.978873
6,EOG,141.160,N/A,0.796238,0.979879,0.509894,0.971831,0.271266,0.979879,0.193584,0.977867,0.977364
7,MPC,100.990,N/A,0.708521,0.965795,0.569276,0.983903,0.304763,0.983903,0.183794,0.973843,0.976861
8,XOM,97.445,N/A,0.745616,0.971831,0.554334,0.979879,0.258889,0.975855,0.186625,0.975855,0.975855
9,PXD,281.210,N/A,0.988265,0.987928,0.527583,0.975855,0.219324,0.95171,0.20553,0.979879,0.973843


In [9]:
position_size = 1000000/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

/Users/zhangxu/opt/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,OXY,67.900,294,1.766426,0.995976,1.093732,1.0,0.723037,1.0,0.215802,0.987928,0.995976
1,DVN,73.831,270,2.019952,0.997988,0.658945,0.987928,0.331095,0.985915,0.281611,1.0,0.992958
2,MRO,31.379,637,1.570223,0.993964,0.762104,0.993964,0.345425,0.987928,0.209224,0.983903,0.98994
3,VLO,128.950,155,0.729904,0.967807,0.786144,0.995976,0.535465,0.997988,0.279869,0.997988,0.98994
4,APA,48.346,413,1.353147,0.991952,0.627307,0.985915,0.361613,0.991952,0.172656,0.971831,0.985412
5,COP,115.000,173,1.115915,0.98994,0.520268,0.973843,0.234812,0.961771,0.221167,0.98994,0.978873
6,EOG,141.160,141,0.796238,0.979879,0.509894,0.971831,0.271266,0.979879,0.193584,0.977867,0.977364
7,MPC,100.990,198,0.708521,0.965795,0.569276,0.983903,0.304763,0.983903,0.183794,0.973843,0.976861
8,XOM,97.445,205,0.745616,0.971831,0.554334,0.979879,0.258889,0.975855,0.186625,0.975855,0.975855
9,PXD,281.210,71,0.988265,0.987928,0.527583,0.975855,0.219324,0.95171,0.20553,0.979879,0.973843
